In [1]:
!pip install kaggle

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import os

In [5]:
os.environ['KAGGLE_CONFIG_DIR'] = '/'

In [6]:
!chmod 600 kaggle.json

In [23]:
!kaggle datasets download -d ichhadhari/indian-birds

Dataset URL: https://www.kaggle.com/datasets/ichhadhari/indian-birds
License(s): CC0-1.0
indian-birds.zip: Skipping, found more recently modified local copy (use --force to force download)


In [24]:
!apt-get install unzip

/bin/bash: apt-get: command not found


In [25]:
!nvidia_smi

/bin/bash: nvidia_smi: command not found


In [26]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [27]:
import torch
torch.cuda.is_available()

ModuleNotFoundError: No module named 'torch'

In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

2024-05-08 13:34:09.208613: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-08 13:34:09.238373: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-08 13:34:09.238975: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-08 13:34:10.476870: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


''

In [30]:
!unzip -qq 'indian-birds.zip' -d 'birds_data'

In [3]:
pip install tensorflow==2.13.1

Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-2.13.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.4 kB)
Using cached tensorflow-2.13.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (479.7 MB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
from IPython.display import HTML

In [52]:
BATCH_SIZE=64 # 32 for model a
IMAGE_SIZE=180
CHANNELS=3

In [53]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory("birds_data/Birds_25/train",
                                                              seed=162,
                                                              shuffle=True,
                                                              image_size=(IMAGE_SIZE, IMAGE_SIZE),
                                                              batch_size=BATCH_SIZE)

Found 30000 files belonging to 25 classes.


In [54]:
val_ds, test_ds = tf.keras.preprocessing.image_dataset_from_directory("birds_data/Birds_25/valid",
                                                              validation_split=0.5,
                                                              subset="both",
                                                              seed=162,
                                                              shuffle=True,
                                                              image_size=(IMAGE_SIZE, IMAGE_SIZE),
                                                              batch_size=BATCH_SIZE)

Found 7500 files belonging to 25 classes.
Using 3750 files for training.
Using 3750 files for validation.


In [55]:
classes = train_ds.class_names

In [56]:
print(classes)

['Asian-Green-Bee-Eater', 'Brown-Headed-Barbet', 'Cattle-Egret', 'Common-Kingfisher', 'Common-Myna', 'Common-Rosefinch', 'Common-Tailorbird', 'Coppersmith-Barbet', 'Forest-Wagtail', 'Gray-Wagtail', 'Hoopoe', 'House-Crow', 'Indian-Grey-Hornbill', 'Indian-Peacock', 'Indian-Pitta', 'Indian-Roller', 'Jungle-Babbler', 'Northern-Lapwing', 'Red-Wattled-Lapwing', 'Ruddy-Shelduck', 'Rufous-Treepie', 'Sarus-Crane', 'White-Breasted-Kingfisher', 'White-Breasted-Waterhen', 'White-Wagtail']


In [57]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
validation_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [30]:
pip install tensorflow==2.13.0 --user

  Using cached tensorflow-2.13.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.4 kB)
Using cached tensorflow-2.13.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (524.2 MB)
Note: you may need to restart the kernel to use updated packages.


In [31]:
import tensorflow as tf
print(tf.__version__)
print(tf.keras.__version__)

2.13.0


AttributeError: module 'tensorflow.keras' has no attribute '__version__'

In [32]:
resize_and_rescale= tf.keras.Sequential([
tf.keras.layers.experimental.preprocessing.Resizing(IMAGE_SIZE,IMAGE_SIZE),
tf.keras.layers.experimental.preprocessing.Rescaling(1.0/255)])

In [33]:
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
  layers.experimental.preprocessing.RandomZoom(0.1),
  layers.experimental.preprocessing.RandomContrast(0.1),
])

In [34]:
train_ds = train_ds.map(
    lambda x, y: (data_augmentation(x, training=True), y)
).prefetch(buffer_size=tf.data.AUTOTUNE)

In [35]:
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = len(classes)

In [36]:
n_classes

25

In [37]:
import keras
import keras.utils

model_a = models.Sequential([
    resize_and_rescale,
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape, kernel_regularizer=keras.regularizers.L1L2()),
    layers.Dropout(0.1),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu', kernel_regularizer=keras.regularizers.L1L2()),
    layers.Dropout(0.2),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128,  kernel_size = (3,3), activation='relu'),
    layers.Dropout(0.3),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.Dropout(0.5),
    layers.GlobalMaxPooling2D(),
    layers.Flatten(),
    layers.Dense(n_classes, activation='softmax'),
])

In [21]:
model_a.build(input_shape=input_shape)

In [18]:
model_a.compile(
    optimizer=keras.optimizers.Adam(weight_decay=0.01),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
history_a = model_a.fit(
    train_ds,
    batch_size=BATCH_SIZE,
    validation_data=validation_ds,
    verbose=1,
    epochs=60,
    callbacks=tf.keras.callbacks.EarlyStopping(monitor = "val_accuracy", # watch the val accuracy metric
                                              patience = 10,
                                              restore_best_weights = True),
)

Epoch 1/60
938/938 [==============================] - 281s 295ms/step - loss: 2.4205 - accuracy: 0.2623 - val_loss: 2.0360 - val_accuracy: 0.4963
Epoch 2/60
938/938 [==============================] - 277s 295ms/step - loss: 1.4639 - accuracy: 0.5655 - val_loss: 1.6855 - val_accuracy: 0.6000
Epoch 3/60
938/938 [==============================] - 278s 297ms/step - loss: 1.1683 - accuracy: 0.6539 - val_loss: 1.4921 - val_accuracy: 0.6888
Epoch 4/60
938/938 [==============================] - 278s 296ms/step - loss: 1.0194 - accuracy: 0.6991 - val_loss: 1.4360 - val_accuracy: 0.7003
Epoch 5/60
938/938 [==============================] - 278s 296ms/step - loss: 0.9249 - accuracy: 0.7245 - val_loss: 1.2601 - val_accuracy: 0.7715
Epoch 6/60
938/938 [==============================] - 277s 295ms/step - loss: 0.8614 - accuracy: 0.7427 - val_loss: 1.2438 - val_accuracy: 0.7347
Epoch 7/60
938/938 [==============================] - 275s 293ms/step - loss: 0.8104 - accuracy: 0.7573 - val_loss: 1.1793 -

In [ ]:
model_a.save('model_a.keras')

In [58]:
pretrained_model = tf.keras.applications.EfficientNetB3(
    input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
    include_top=False,
    weights='imagenet',
    pooling='max'
)

In [59]:
for i, layer in enumerate(pretrained_model.layers):
    pretrained_model.layers[i].trainable = False
    
model_b = models.Sequential([
    tf.keras.layers.experimental.preprocessing.Resizing(IMAGE_SIZE,IMAGE_SIZE),
    pretrained_model,
    layers.Dense(256, activation="relu"),
    layers.BatchNormalization(),
    layers.Dropout(0.2),    
    layers.Dense(n_classes, activation='softmax'),
])

In [60]:
model_b.build(input_shape=input_shape)

In [61]:
model_b.compile(
    optimizer=keras.optimizers.Adam(weight_decay=0.01),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [62]:
history_b = model_b.fit(
    train_ds,
    batch_size=BATCH_SIZE,
    validation_data=validation_ds,
    verbose=1,
    epochs=30,
    callbacks=tf.keras.callbacks.EarlyStopping(monitor = "val_accuracy", # watch the val accuracy metric
                                              patience = 20,
                                              restore_best_weights = True),
)

Epoch 1/30
469/469 [==============================] - 423s 878ms/step - loss: 0.4458 - accuracy: 0.8703 - val_loss: 0.2659 - val_accuracy: 0.9208
Epoch 2/30
469/469 [==============================] - 420s 897ms/step - loss: 0.2316 - accuracy: 0.9303 - val_loss: 0.2398 - val_accuracy: 0.9304
Epoch 3/30
469/469 [==============================] - 415s 885ms/step - loss: 0.1855 - accuracy: 0.9437 - val_loss: 0.1984 - val_accuracy: 0.9440
Epoch 4/30
469/469 [==============================] - 407s 868ms/step - loss: 0.1564 - accuracy: 0.9510 - val_loss: 0.2266 - val_accuracy: 0.9339
Epoch 5/30
469/469 [==============================] - 409s 873ms/step - loss: 0.1381 - accuracy: 0.9581 - val_loss: 0.2060 - val_accuracy: 0.9368
Epoch 6/30
469/469 [==============================] - 426s 908ms/step - loss: 0.1237 - accuracy: 0.9631 - val_loss: 0.1879 - val_accuracy: 0.9477
Epoch 7/30
469/469 [==============================] - 430s 918ms/step - loss: 0.1193 - accuracy: 0.9622 - val_loss: 0.1762 -

INFO:tensorflow:Assets written to: model_b/assets


In [64]:
model_b.save('model_b.keras')

In [65]:
model_b.evaluate(test_ds)

59/59 [==============================] - 50s 846ms/step - loss: 0.1633 - accuracy: 0.9557


[0.16327223181724548, 0.9557333588600159]